# Manual import of rlc json file from open data portal

https://www.postgresqltutorial.com/postgresql-python/query/

In [55]:
import os
from psycopg2 import connect
import configparser
import numpy as np

import requests
from psycopg2.extras import execute_values

import pandas as pd
import pandas.io.sql as pandasql

import csv

In [51]:
CONFIG=configparser.ConfigParser()
CONFIG.read('/home/cnangini/db.cfg')
dbset=CONFIG['DBSETTINGS']
conn=connect(**dbset)
i=0

# Functions

In [52]:
def pull_rlc(conn):
  '''
  Connect to bigdata RDS, pull Red Light Camera json file from Open Data API,
  and overwrite existing rlc table in the vz_safety_programs_staging schema.
  '''

  local_table='vz_safety_programs_staging.rlc_manual'

  url = "https://secure.toronto.ca/opendata/cart/red_light_cameras.json"
  return_json = requests.get(url).json()
  rows = [list(feature.values()) for feature in return_json]
  insert = 'INSERT INTO {0} VALUES %s'.format(local_table)
  with conn:
    with conn.cursor() as cur:
        print(cur)
        print(insert)
        print(rows)
        execute_values(cur, insert, rows)
        print(rows)

In [47]:
def pull_rlc_csv(conn):
  '''
  Connect to bigdata RDS, pull Red Light Camera json file from Open Data API,
  and overwrite existing rlc table in the vz_safety_programs_staging schema.
  '''

  local_table='vz_safety_programs_staging.rlc_fromcsv'

  url = "https://secure.toronto.ca/opendata/cart/red_light_cameras.csv"

  with requests.Session() as s:
    download = s.get(url)
    print(download)
    
    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        if row != ['RLC']:
            print(row)
            
   cur = conn.cursor()
   insert_query = "INSERT INTO users VALUES {}".format("(10, 'hello@dataquest.io', 'Some Name', '123 Fake St.')")
   cur.execute(insert_query)
   conn.commit()

# Truncate view, pull from open data, and check

In [49]:
# pull_rlc_csv(conn)

## Truncate first

In [63]:
query='''
TRUNCATE vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
cur.close()

In [64]:
# Check size

query='''
SELECT COUNT(*) FROM vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
row = cur.fetchone()
print(row)

cur.close()

(0,)


## Pull from open data

In [65]:
pull_rlc(conn)

<cursor object at 0x7fdd48ef0520; closed: 0>
INSERT INTO vz_safety_programs_staging.rlc_manual VALUES %s
[[None, None, None, '0223', '6001', '51', None, 315752.39, '2007-11-09', None, None, 'Toronto Centre(13)', None, 'Toronto and East York', -79.364023, 'PARLIAMENT ST', 43.654559, None, None, 'Richmond St E And Parliament St', 4834744.176, 'RICHMOND ST E'], [None, None, None, '0205', '6002', '52', None, 314395.743, '2007-11-09', None, None, 'Spadina-Fort York(10)', None, 'Toronto and East York', -79.380868, 'YORK ST', 43.64146, 'WB LAKE SHORE BLVD', None, 'Lake Shore Blvd W And York St', 4833286.82, 'LAKE SHORE BLVD W'], [None, None, 'SHOPPING CENTRE', '1564', '6003', '32', None, 309002.26, '2007-11-09', None, None, 'York Centre(6)', 'PRIVATE ACCESS', 'North York', -79.447589, 'CARPENTER RD', 43.792013, None, None, 'Steeles Ave W And Carpenter Rd / Private Access @ Shopping Centre', 4850007.08, 'STEELES AVE W'], [None, None, None, '1115', '6004', '32', None, 310476.393, '2007-11-09', 

## Change owner

In [66]:
query='''
ALTER TABLE vz_safety_programs_staging.rlc_manual
    OWNER to vz_admins;
'''

cur = conn.cursor()
cur.execute(query)
cur.close()

## Count rows in updated table

In [68]:
query='''
SELECT COUNT(*) FROM vz_safety_programs_staging.rlc_manual;
'''

cur = conn.cursor()
cur.execute(query)
row = cur.fetchone()
print(row)

cur.close()

(148,)
